In [ ]:
!pip install deeplake

In [ ]:
import deeplake
from PIL import Image
import numpy as np
import os

token = ""

In [ ]:
ds = deeplake.empty('hub://um_project/art-shap', token=token)

In [ ]:
test_ds = deeplake.load('hub://um_project/art-test', token=token, read_only=True)

In [ ]:
classes_labels = test_ds.labels.info.class_names
len(classes_labels)

In [ ]:
class_images = {label: [] for label in classes_labels}

shap_test_subset_indexes = [2010, 2029, 2001, 1960, 2257, 2912, 1179, 2068, 2274, 2208, 2801, 3852, 553]

# Iterate through the dataset and store 7 images for each class
for i, sample in enumerate(test_ds):
    label = sample['labels'].data()['text'][0]  # Access the first element of the list
    if i in shap_test_subset_indexes:
        image_array = sample['images'].data()['value']
        class_images[label].append(image_array)

In [ ]:
from matplotlib import pyplot as plt

# Create a grid of subplots and display the images
num_classes = len(classes_labels)
num_images_per_class = 1
fig = plt.figure(figsize=(28, 56))

for i, class_label in enumerate(classes_labels):
    for j, image_array in enumerate(class_images[class_label]):
        ax = fig.add_subplot(num_classes, num_images_per_class, i * num_images_per_class + j + 1)
        ax.imshow(image_array)
        ax.axis('off')

        # Set the class name above the first image in each row
        if j == int(num_images_per_class / 2):
            ax.set_title(class_label, fontsize=32, ha='center', va='center', weight='bold')

plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
with ds:
    # Create tensors
    ds.create_tensor('images', htype = 'image', sample_compression = 'jpeg')
    ds.create_tensor('labels', htype = 'class_label', class_names = classes_labels)

In [ ]:
with ds:
    # Iterate through data and append to Deep Lake dataset
    for label in class_images:
        ds.append({'images': class_images[label][0], 'labels': label})